<a href="https://colab.research.google.com/github/Praveen-Prabhu/code-generation/blob/main/code-generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

# Install required libraries
!pip install -q -U transformers accelerate bitsandbytes unsloth vllm wandb
!pip install --upgrade pillow

import os

### HuggingFace & W&B Authentication

In [2]:
from huggingface_hub import login, HfApi, HfFolder
import wandb
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')
WANDB_TOKEN = userdata.get('WANDB_TOKEN')

login(token=HF_TOKEN)
wandb.login(key=WANDB_TOKEN)

wandb.init(
    project="codegen-llama-3.1-8b-grpo",
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: praveenprabhu to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
# Define Hugging Face Repository for Saving Checkpoints
HF_REPO = "PraveenPrabhu/codegen-llama-3.1-8b-grpo"

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [4]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Load up `Llama 3.1 8B Instruct`, and set parameters

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 03-16 14:22:47 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.43%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 2.59 GB. Also swap space = 2 GB.
WARNING 03-16 14:22:55 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-16 14:23:09 config.py

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

INFO 03-16 14:25:56 model_runner.py:1562] Graph capturing finished in 41 secs, took 0.59 GiB
INFO 03-16 14:25:56 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 56.38 seconds


tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.3.14 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [6]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# # uncomment middle messages for 1-shot prompting
# def get_gsm8k_questions(split = "train") -> Dataset:
#     data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
#     data = data.map(lambda x: { # type: ignore
#         'prompt': [
#             {'role': 'system', 'content': SYSTEM_PROMPT},
#             {'role': 'user', 'content': x['question']}
#         ],
#         'answer': extract_hash_answer(x['answer'])
#     }) # type: ignore
#     return data # type: ignore

# dataset = get_gsm8k_questions()

# uncomment middle messages for 1-shot prompting
def get_questions(split = "train") -> Dataset:
    data = load_dataset('MatrixStudio/Codeforces-Python-Submissions')[split] # type: ignore
    data = data.filter(lambda x: x["verdict"] == "OK")
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['prompt']}
        ],
        'answer': x['response']
    }) # type: ignore
    return data # type: ignore

dataset = get_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md:   0%|          | 0.00/1.75k [00:00<?, ?B/s]

train-00000-of-00009.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

train-00001-of-00009.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

train-00002-of-00009.parquet:   0%|          | 0.00/167M [00:00<?, ?B/s]

train-00003-of-00009.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/621356 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/69040 [00:00<?, ? examples/s]

Filter:   0%|          | 0/621356 [00:00<?, ? examples/s]

Map:   0%|          | 0/274883 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [7]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    # max_steps = 1,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "wandb",  # Enable Weights & Biases Tracking
    output_dir = "checkpoints",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [8]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
# trainer.train()

In [9]:
# Load Checkpoint from Hugging Face if Exists
from huggingface_hub import hf_hub_download
import shutil

def download_checkpoint(repo_id, filename, local_dir="hf_checkpoints"):
    """Download a checkpoint file from Hugging Face and return its local path."""
    os.makedirs(local_dir, exist_ok=True)
    try:
        return hf_hub_download(repo_id, filename, local_dir=local_dir)
    except:
        return None  # Return None if checkpoint does not exist

lora_path = download_checkpoint(HF_REPO, "lora_adapter.safetensors")
optimizer_path = download_checkpoint(HF_REPO, "optimizer_state.pth")
trainer_path = download_checkpoint(HF_REPO, "trainer_state.pth")
tokenizer_path = download_checkpoint(HF_REPO, "tokenizer.json")

if lora_path and optimizer_path and trainer_path and tokenizer_path:
    model.load_lora(lora_path)
    trainer.load_state_dict(torch.load(trainer_path))
    trainer.optimizer.load_state_dict(torch.load(optimizer_path))
    tokenizer = torch.load(tokenizer_path)
    print("Resumed training from Hugging Face checkpoint!")

optimizer_state.pth:   0%|          | 0.00/171M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.99M [00:00<?, ?B/s]

In [ ]:
# Train Model
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 274,883 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 83,886,080/8,000,000,000 (1.05% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


-------------------- Question:
Title: none

Time Limit: None seconds

Memory Limit: None megabytes

Problem Description:
This is an interactive problem.

Natasha is going to fly to Mars. Finally, Natasha sat in the rocket. She flies, flies... but gets bored. She wishes to arrive to Mars already! So she decides to find something to occupy herself. She couldn't think of anything better to do than to calculate the distance to the red planet.

Let's define $x$ as the distance to Mars. Unfortunately, Natasha does not know $x$. But it is known that $1 \le x \le m$, where Natasha knows the number $m$. Besides, $x$ and $m$ are positive integers.

Natasha can ask the rocket questions. Every question is an integer $y$ ($1 \le y \le m$). The correct answer to the question is $-1$, if $x&lt;y$, $0$, if $x=y$, and $1$, if $x&gt;y$. But the rocket is broken — it does not always answer correctly. Precisely: let the correct answer to the current question be equal to $t$, then, if the rocket answers th

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,-0.000000,-0.053333,0.260950,184.833344,0.000000,-0.053333,0.000000,0.000000,0.000000,0.000000
2,-0.000000,-0.266000,0.651564,475.666687,0.000000,-0.266000,0.000000,0.000000,0.000000,0.000000
3,0.000000,-0.165000,0.404166,310.000000,0.000006,-0.165000,0.000000,0.000000,0.000000,0.000000
4,0.000000,-0.432833,0.746778,359.833344,0.000010,-0.432833,0.000000,0.000000,0.000000,0.000000
5,0.000000,-0.011167,0.027353,309.333344,0.000007,-0.011167,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.062500,0.068465,134.000000,0.000007,0.062500,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.041667,0.102062,315.000000,0.000007,0.041667,0.000000,0.000000,0.000000,0.000000
8,0.000000,-0.412167,0.751521,441.500000,0.000005,-0.412167,0.000000,0.000000,0.000000,0.000000
9,0.000000,-0.251333,0.289064,214.166672,0.000009,-0.251333,0.000000,0.000000,0.000000,0.000000
10,0.000000,-0.064000,0.132091,221.833344,0.000009,-0.064000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Title: Game

Time Limit: None seconds

Memory Limit: None megabytes

Problem Description:
Two players play a simple game. Each player is provided with a box with balls. First player's box contains exactly *n*1 balls and second player's box contains exactly *n*2 balls. In one move first player can take from 1 to *k*1 balls from his box and throw them away. Similarly, the second player can take from 1 to *k*2 balls from his box in his move. Players alternate turns and the first player starts the game. The one who can't make a move loses. Your task is to determine who wins if both players play optimally.

Input Specification:
The first line contains four integers *n*1,<=*n*2,<=*k*1,<=*k*2. All numbers in the input are from 1 to 50.

This problem doesn't have subproblems. You will get 3 points for the correct submission.

Output Specification:
Output "First" if the first player wins and "Second" otherwise.

Demo Input:
['2 2 1 2\n', '2 1 1 1\n']

Demo Output:

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,-0.000000,-0.053333,0.260950,184.833344,0.000000,-0.053333,0.000000,0.000000,0.000000,0.000000
2,-0.000000,-0.266000,0.651564,475.666687,0.000000,-0.266000,0.000000,0.000000,0.000000,0.000000
3,0.000000,-0.165000,0.404166,310.000000,0.000006,-0.165000,0.000000,0.000000,0.000000,0.000000
4,0.000000,-0.432833,0.746778,359.833344,0.000010,-0.432833,0.000000,0.000000,0.000000,0.000000
5,0.000000,-0.011167,0.027353,309.333344,0.000007,-0.011167,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.062500,0.068465,134.000000,0.000007,0.062500,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.041667,0.102062,315.000000,0.000007,0.041667,0.000000,0.000000,0.000000,0.000000
8,0.000000,-0.412167,0.751521,441.500000,0.000005,-0.412167,0.000000,0.000000,0.000000,0.000000
9,0.000000,-0.251333,0.289064,214.166672,0.000009,-0.251333,0.000000,0.000000,0.000000,0.000000
10,0.000000,-0.064000,0.132091,221.833344,0.000009,-0.064000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Title: Free Ice Cream

Time Limit: None seconds

Memory Limit: None megabytes

Problem Description:
After their adventure with the magic mirror Kay and Gerda have returned home and sometimes give free ice cream to kids in the summer.

At the start of the day they have *x* ice cream packs. Since the ice cream is free, people start standing in the queue before Kay and Gerda's house even in the night. Each person in the queue wants either to take several ice cream packs for himself and his friends or to give several ice cream packs to Kay and Gerda (carriers that bring ice cream have to stand in the same queue).

If a carrier with *d* ice cream packs comes to the house, then Kay and Gerda take all his packs. If a child who wants to take *d* ice cream packs comes to the house, then Kay and Gerda will give him *d* packs if they have enough ice cream, otherwise the child will get no ice cream at all and will leave in distress.

Kay wants to find the amount of i

In [ ]:
# Save All States After Training
# torch.save(trainer.state_dict(), "trainer_state.pth")
model.save_pretrained_merged("lora_adapter", tokenizer, save_method = "lora",)
torch.save(tokenizer, "tokenizer.json")
torch.save(trainer.optimizer.state_dict(), "optimizer_state.pth")

In [ ]:
# Finish W&B Logging
wandb.finish()

In [ ]:
# Upload All States to Hugging Face for Persistence
api = HfApi()

# api.upload_file(
#     path_or_fileobj="lora_adapter.safetensors",
#     path_in_repo="lora_adapter.safetensors",
#     repo_id=HF_REPO,
#     token=HF_TOKEN,
# )

model.push_to_hub_merged(HF_REPO, tokenizer, save_method = "lora")

# api.upload_file(
#     path_or_fileobj="trainer_state.pth",
#     path_in_repo="trainer_state.pth",
#     repo_id=HF_REPO,
#     token=HF_TOKEN,
# )

api.upload_file(
    path_or_fileobj="optimizer_state.pth",
    path_in_repo="optimizer_state.pth",
    repo_id=HF_REPO,
    token=HF_TOKEN,
)

api.upload_file(
    path_or_fileobj="tokenizer.json",
    path_in_repo="tokenizer.json",
    repo_id=HF_REPO,
    token=HF_TOKEN,
)

<a name="Inference"></a>
### Inference

Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "WHAT IS GREATER 9.11 OR 9.9"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
# model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "WHAT IS GREATER 9.11 OR 9.9"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("lora_adapter"),
)[0].outputs[0].text

output

In [ ]:
print(output)

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)